## Aligning serial sections of adult mouse brain (10x Visium)

In this notebook, we will illustrate how to perform the alignment of 2 serial sections from a Visium (10x) mouse brain example dataset.

### Prerequisites

First of all, you must have a Python environment with [ipykernel](https://ipython.readthedocs.io/en/stable/install/kernel_install.html) for running this notebook, or [Jupyter](https://jupyter.org/).

Alternatively, you can run this notebook in [Google Colab]().

### Installing dependencies

We will install STIM (both the Java-based components, and the Python bindings), as well as other dependencies that might be useful for later downstream analysis (i.e., `scanpy`)

In [ ]:
if 'google.colab' in str(get_ipython()):
  !apt-get install openjdk-8-jdk maven
  !git clone https://github.com/danilexn/stim.git
  !cd stim && git checkout stim_logging && ./install.sh -i ./bin
  !pip install git+https://github.com/danilexn/stimwrap@stim_commands
  !pip install scanpy
else:
  !conda install -c conda-forge stim
  !pip install stimwrap
  !pip install scanpy

### Downloading the data

We will download the zip file containing the Visium data. These example data is available at [Google Drive](https://drive.google.com/file/d/1qzzu4LmRukHBvbx_hiN2FOmIladiT7xx/view). With the code below, we will automatically download it.

In [ ]:
!pip install gdown

In [ ]:
import gdown

# a file
url = "https://drive.google.com/uc?id=1qzzu4LmRukHBvbx_hiN2FOmIladiT7xx"
output = "visium.zip"
gdown.download(url, output)

Downloading (and decompressing) the whole dataset should take takes < 1 minute with a ~200Mbps internet connection.

### Importing `stimwrap`

In general, `STIM` is a console-based tool.

However, when running your analysis in the `Python` ecosystem (like here), you can transparently run `STIM` from `Python` by leveraging the wrapper `stimwrap`.

`stimwrap` provides bindings for all commands, and additional tools for data preprocessing and conversion prior to downstream analysis.

In [ ]:
import stimwrap as st

In [ ]:
# specifically in Google Colab
st.set_bin_path("./stim/bin")

### Creating a N5 container

STIM requires that all sections are resaved into a single N5 container, a single directory containing all data (spatial expression values), metadata (cell annotations), and the output from STIM registration (landmarks, transformation matrices).

In [ ]:
container_path = "./visium.n5"
sections = ['slice1.h5ad', 'slice2.h5ad']
sections_numbers = [1, 2] # for later setting the Z coordinates

st.resave(input="visium.zip/section1_locations.csv,visium.zip/section1_reads.csv,slice1.h5ad", container=container_path)
st.resave(input="visium.zip/section2_locations.csv,visium.zip/section2_reads.csv,slice2.h5ad", container=container_path)

### Pairwise registration

As indicated in its name, STIM will handle ST data as images. These are multi-channel images where the XY dimensions can be specified by a scaling factor (e.g., 1:1 to map 1 pixel to 1 ST unit), and the channels are genes.

During pairwise registration, STIM will automatically find corresponding points between pairs of sections for a subset of genes, and keep those with _high quality/agreement_ across all genes for a pair of sections. This is required prior to assembling a global alignment model (when more than 2 sections are provided).

A subset of genes is used to avoid registering with all genes (in sequencing-based ST, this can lead to ~30,000 channels). This might be too time-consuming, and also most genes do not have sufficient information to render images with spatial patterns that can be used for feature detection (sparsity problem). By default, STIM detects genes with highest variance as a proxy for genes that might show suitable spatial patterns. Otherwise, the user can specify a set of genes used to render images for pairwise alignment. In this tutorial, we use STIM for computing these variability metrics.

In [ ]:
st.align_pairs(container=container_path,
               num_genes = 10,
               skip = 10,
               max_epsilon = 0,
               range = 2,
               scale = 0.05, # recommended for Visium data
               num_threads = 2) # you can adapt depending on your Colab

**Importantly**, if you try to run pairwise alignment more than once, you need to specify the argument `overwrite=True` when calling `st.align_pairs`.

Otherwise, `STIM` assumes that pairwise alignment was performed and will exit.

### Visualization of results (**not in Google Colab!!!!**)

It is good practice to manually assess the results of pairwise alignment before proceeding or using these data for analysis, as the set of parameters used for registration might have not been suitable for the data at hand. Some reasons leading to poor alignment might be:

- Poor selection of the subset of genes used for alignment
- Scale (or render factor) parameter too large or too small
- Poor selection of alignment error (`--maxEpsilon`) parameter
- Data is too noisy and might need some filtering (e.g., with `--ffMedian` or `--ffSingleSpot`)

STIM provides GUI-based tools to interactively assess the result from pairwise alignment:

In [ ]:
st.explorer(container=container_path,
            datasets=sections)

Alternatively, you can use an interactive pairwise alignment tool to find suitable parameters, iteratively

In [ ]:
st.align_interactive(container=container_path,
                     section_a=sections[0],
                     section_b=sections[1])

To run the interactive alignment via `st.align_interactive` or `st.explorer`,
make sure you are running this notebook in a computer with a graphical environment, or
that you are doing redirection of the window server (e.g., X11 redirection via `ssh -X ...`).

You can learn more about this [here](https://goteleport.com/blog/x11-forwarding/).

### Global alignment

Once you are satisfied with the results from the pairwise alignment of pairs of sections, you can proceed with the global alignment.

This last step optimizes a global model taking into account all pairs of keypoints.

This reduces error propagation across sections, which might lead to very large distortions in the reconstruction.

In [ ]:
st.align_global(container=container_path,
                skip_icp=True)

### Visualization of results (**not in Google Colab!!!!**)

Similarly as before, it is good practice to visualize the results after the global alignment procedure.

STIM can leverage `BigDataViewer` for 3D visualization of these data:

In [ ]:
st.bdv_view3d(container=container_path,
              genes=['Calm2', 'Mbp'])

To run `BigDataViewer`, similarly to above, make sure you are running this notebook in a
computer with a graphical environment, or that you are doing redirection of the window server
(e.g., X11 redirection via `ssh -X ...`).

### Storing the 3D coordinates in `AnnData`

Prior to analysing these objects with `scanpy` or other tools from the `scverse` ecosystem, you can apply the transformation model
and store the transformed 3D coordinates as a new layer in the `AnnData` (or `N5`) objects.

In [ ]:
# load N5 container with stimwrap
container = st.Container(container_path)

In [ ]:
# iterate over datasets and apply the computed transformation
for z_axis, dataset_name in zip(sections_numbers, container.get_dataset_names()):
    with container.get_dataset(dataset_name, mode="r+") as dataset:
        dataset.apply_save_transform(transformation="model_sift",
                                     locations='spatial',
                                     destination='spatial_transform_sift',
                                     z_coord=z_axis)

### Demo: interoperability with `scanpy`

Here, we showcase the interoperability of STIM (via `AnnData`-backed N5) by plotting genes and running some data processing with `scanpy`.

First of all, you can create a single `AnnData` object that can be loaded at once with `scanpy` (all cells in the same file).

In [ ]:
import scanpy as sc

In [ ]:
import anndata as ad

adata_concatenated = ad.concat([ad.read_h5ad(adata_path) for adata_path in sections], join='inner', index_unique="_")
adata_concatenated.write_h5ad("./mouse_visium_aligned.h5ad")

##### Note

If you get errors in the cell above because of `__DATA_TYPES__` or because of `column-order`, run the following command:

In [ ]:
# run if necessary (comment out line below)
# container.cleanup_container()

#### Plotting gene expression

We transpose the Z-axis coordinates for plotting with scanpy (from a different point of view)

In [ ]:
adata_concatenated.obsm['spatial_transform_sift_plotting'] = adata_concatenated.obsm['spatial_transform_sift'][:, [2, 0, 1]]

We use the `pl.embedding` function for faster plotting (also, axes are scaled to the same magnitude). One can alternatively use the `pl.spatial` function

In [ ]:
sc.pl.embedding(adata_concatenated, color=['IGKC'], projection='3d', size=1, basis='spatial_transform_sift_plotting', cmap='Reds')

#### Plotting normalized gene expression

In the plot above, raw counts are shown. As different sections had different sequencing depth, the intensities are not fully comparable.

For improved visualization (and downstream analysis), we can normalize the values across sections. This follows the typical scanpy workflow.

In [ ]:
# Filter and normalize
sc.pp.calculate_qc_metrics(adata_concatenated, inplace=True)
sc.pp.filter_cells(adata_concatenated, min_counts=250)
sc.pp.filter_cells(adata_concatenated, max_counts=10000)
sc.pp.normalize_total(adata_concatenated, inplace=True)
sc.pp.log1p(adata_concatenated)

Now we can plot again, which will show depth and log-normalized counts.

In [ ]:
sc.pl.embedding(adata_concatenated, color=['IGKC'], projection='3d', size=1, basis='spatial_transform_sift_plotting', cmap='Reds')

From here we can proceed with downstream analysis, like cell type clustering, differential expression, discovery of spatial features...